In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# default_exp decoder

In [3]:
#hide
from nbdev.showdoc import *
from fastcore.test import *


In [4]:
#export
import pandas as pd # package for high-performance, easy-to-use data structures and data analysis
import numpy as np # fundamental package for scientific computing with Python

from fastai2.basics import *
from fastai2.text.all import *
from fastai2.callback.all import *
import pickle
import os
import torch
import numpy as np
from scipy.stats import spearmanr

In [5]:
#export
data_path = Path("/home/mrdbarros/projetos/data/gquest_data/")

In [6]:
#export
print('Reading data...')
train_data = pd.read_csv(data_path/'train/train.csv')
test_data = pd.read_csv(data_path/'test/test.csv')
sample_submission = pd.read_csv(str(data_path/'sample_submission.csv'))
print('Reading data completed')

Reading data...
Reading data completed


In [7]:
#export
train_data.columns
target_columns = train_data.columns[11:]
target_columns

Index(['question_asker_intent_understanding', 'question_body_critical',
       'question_conversational', 'question_expect_short_answer',
       'question_fact_seeking', 'question_has_commonly_accepted_answer',
       'question_interestingness_others', 'question_interestingness_self',
       'question_multi_intent', 'question_not_really_a_question',
       'question_opinion_seeking', 'question_type_choice',
       'question_type_compare', 'question_type_consequence',
       'question_type_definition', 'question_type_entity',
       'question_type_instructions', 'question_type_procedure',
       'question_type_reason_explanation', 'question_type_spelling',
       'question_well_written', 'answer_helpful',
       'answer_level_of_information', 'answer_plausible', 'answer_relevance',
       'answer_satisfaction', 'answer_type_instructions',
       'answer_type_procedure', 'answer_type_reason_explanation',
       'answer_well_written'],
      dtype='object')

In [8]:
#export
with open(data_path/'vocab.pkl', 'rb') as vocab_file:
 
    # Step 3
    lm_vocab = pickle.load(vocab_file)
 
    # After config_dictionary is read from file

In [9]:
#export
text_columns=['question_title', 'question_body', 'question_user_name',
       'question_user_page', 'answer', 'answer_user_name', 'answer_user_page',
       'url', 'category', 'host']


In [10]:
train_data=train_data[:100]

In [11]:
#export
splits = RandomSplitter()(train_data)

In [12]:
#export
df_tokenized,token_count=tokenize_df(train_data,text_columns)
x_tfms = [attrgetter("text"), Numericalize(vocab=lm_vocab)]


In [13]:
class DataFrameToTensor(Transform):
    def __init__(self,columns): self.columns=columns
    def encodes(self,o): return tensor(o[self.columns].values.astype(float))
    def decodes(self,x): return x

In [14]:
dsrc = DataSource(df_tokenized ,splits=splits, tfms=[x_tfms, [DataFrameToTensor(target_columns)]], dl_type=SortedDL)

In [15]:
#export
torch.cuda.empty_cache()

In [16]:
#export
dbunch = dsrc.databunch(before_batch=pad_input,bs=2)


In [17]:
x_train,y_train=dbunch.train_dl.one_batch()
x_valid,y_valid=dbunch.valid_dl.one_batch()
print(type(x_valid))
print(type(y_valid))

<class 'fastai2.text.data.TensorText'>
<class 'torch.Tensor'>


In [18]:
print(x_train.shape,len(y_train[1]),len(y_train))
print(x_valid.shape,len(y_valid[1]),len(y_valid))
print(x_valid[1].shape)
print(len(y_valid[1]))

torch.Size([2, 1891]) 30 2
torch.Size([3, 1570]) 30 3
torch.Size([1570])
30


In [19]:
#export
#dbunch.show_batch(max_n=2,trunc_at=60)


In [20]:
#export
torch.cuda.empty_cache()

In [54]:
spearman_corr_coef(full_predict,full_target)

> <ipython-input-53-829500e03ac6>(7)spearman_corr_coef()
-> coef, p = spearmanr(a[:,i], b[:,i])
(Pdb) n
> <ipython-input-53-829500e03ac6>(8)spearman_corr_coef()
-> full_coef +=coef
(Pdb) n
> <ipython-input-53-829500e03ac6>(9)spearman_corr_coef()
-> print("i:",i,"full_coef",full_coef)
(Pdb) n
i: 0 full_coef -0.05434409264771256
> <ipython-input-53-829500e03ac6>(5)spearman_corr_coef()
-> for i in range(n_col):
(Pdb) n
> <ipython-input-53-829500e03ac6>(6)spearman_corr_coef()
-> pdb.set_trace()
(Pdb) n
> <ipython-input-53-829500e03ac6>(7)spearman_corr_coef()
-> coef, p = spearmanr(a[:,i], b[:,i])
(Pdb) n
> <ipython-input-53-829500e03ac6>(8)spearman_corr_coef()
-> full_coef +=coef
(Pdb) n
> <ipython-input-53-829500e03ac6>(9)spearman_corr_coef()
-> print("i:",i,"full_coef",full_coef)
(Pdb) n
i: 1 full_coef 0.23181547089183668
> <ipython-input-53-829500e03ac6>(5)spearman_corr_coef()
-> for i in range(n_col):
(Pdb) (i)
1
(Pdb) n
> <ipython-input-53-829500e03ac6>(6)spearman_corr_coef()
-> pdb.s

/home/mrdbarros/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/mrdbarros/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/home/mrdbarros/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/home/mrdbarros/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/home/mrdbarros/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


> <ipython-input-53-829500e03ac6>(8)spearman_corr_coef()
-> full_coef +=coef
(Pdb) (coef)
nan
(Pdb) (spearmanr(a[:,i], b[:,i]))
SpearmanrResult(correlation=nan, pvalue=nan)
(Pdb) (a[:,i])
tensor([ 0.3674,  0.6421,  0.4983,  0.3000, -0.0271,  0.0221, -0.1050,  0.6450,
        -0.0249, -0.0215,  0.0778,  0.6992,  0.6245,  0.0195,  0.1265,  0.1532,
         0.4062,  0.6157,  0.4568,  0.0855])
(Pdb) (b[:,i])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
(Pdb) c
i: 9 full_coef nan
> <ipython-input-53-829500e03ac6>(6)spearman_corr_coef()
-> pdb.set_trace()
(Pdb) c
i: 10 full_coef nan
> <ipython-input-53-829500e03ac6>(7)spearman_corr_coef()
-> coef, p = spearmanr(a[:,i], b[:,i])
(Pdb) c
i: 11 full_coef nan
> <ipython-input-53-829500e03ac6>(6)spearman_corr_coef()
-> pdb.set_trace()
(Pdb) c
i: 12 full_coef nan
> <ipython-input-53-829500e03ac6>(7)spearman_corr_coef()
-> coef, p = spearmanr(a[:,i], b[:,i])
--KeyboardInterrupt--
(Pdb) c
i: 13 full_coef na

nan

In [53]:
def spearman_corr_coef(a,b):
    full_coef=0.
    #pdb.set_trace()
    n_col=a.shape[1]
    for i in range(n_col):
        #pdb.set_trace()
        coef, p = spearmanr(a[:,i], b[:,i])
        full_coef +=coef
    print("n_col",float(n_col))
    print("full_coef",full_coef)
    return full_coef/float(n_col)

In [23]:
#export
#learn = text_classifier_learner(dbunch, AWD_LSTM, metrics=[accuracy], path=data_path,drop_mult=0.5)
learn = text_regression_learner(dbunch, AWD_LSTM,seq_len=10, path=data_path,drop_mult=0.5).to_fp16()

In [24]:
#export
learn = learn.load_encoder('enc1')


In [25]:
#export
len(target_columns)

30

In [26]:
#export
#pdb.set_trace()
learn.fit_one_cycle(4, moms=(0.8,0.7,0.8))

epoch     train_loss  valid_loss  time    
0         0.840758    0.469052    00:18     
1         0.786299    0.491357    00:19     
2         0.732889    0.497470    00:19     
3         0.681953    0.517493    00:18     


In [57]:
learn.save("first_decoder")

In [84]:
learn.load("first_decoder")

In [24]:
learn.unfreeze()
learn.opt = learn.create_opt()
learn.fit_one_cycle(8, slice(1e-5,1e-3))

epoch,train_loss,valid_loss,time


RuntimeError: CUDA out of memory. Tried to allocate 48.00 MiB (GPU 0; 3.94 GiB total capacity; 2.77 GiB already allocated; 15.62 MiB free; 247.79 MiB cached)

In [86]:
x,y in dbunch.valid_ds[0]

RuntimeError: The size of tensor a (255) must match the size of tensor b (30) at non-singleton dimension 0

In [27]:
full_predict=None
i=1
print("tamanho:",len(dbunch.valid_ds),"\n")
for x,y in dbunch.valid_ds:
    if  full_predict is None:
        full_predict=learn.predict(x)[0].unsqueeze(dim=0)
        full_target=y.unsqueeze(dim=0)
    else:
        full_predict=torch.cat((full_predict,learn.predict(x)[0].unsqueeze(dim=0)))
        full_target=torch.cat((full_target,y.unsqueeze(dim=0)))
    print(i)
    i=i+1

tamanho: 20 

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [40]:
print(full_target[:,0])
print(full_predict[:,0])

tensor([1.0000, 1.0000, 0.5556, 0.8889, 0.8889, 1.0000, 0.8889, 1.0000, 1.0000,
        0.8889, 1.0000, 0.8889, 0.8889, 0.8333, 1.0000, 1.0000, 0.8889, 1.0000,
        1.0000, 0.7778])
tensor([-0.2206,  0.6313,  0.0508,  0.7026, -0.1221,  0.3962, -0.3396,  0.1775,
        -0.4016,  0.5615,  0.0323,  0.5073,  0.7290,  0.5127,  0.3516,  0.3015,
         0.2112,  0.6333,  0.2668,  0.1365])


In [ ]:
spearman_corr_coef(full_predict,full_target)

In [45]:
full_predict[0]

tensor([ 8.7402e-01,  5.3662e-01, -3.0502e-02,  6.9482e-01,  8.1592e-01,
         9.0332e-01,  5.7568e-01,  4.3042e-01,  2.7173e-01, -6.9141e-04,
         3.9014e-01,  3.1274e-01,  3.8666e-02,  9.6588e-03, -1.3489e-02,
         6.0852e-02,  6.5381e-01,  1.7786e-01,  3.2446e-01, -2.8229e-03,
         7.9395e-01,  9.2285e-01,  6.4551e-01,  9.3994e-01,  9.6875e-01,
         8.5889e-01,  5.8447e-01,  1.6052e-01,  4.9707e-01,  9.2188e-01])

In [47]:
full_predict[0]-full_target[0]

tensor([-0.1260, -0.0189, -0.0305,  0.0282,  0.1493, -0.0967,  0.1312,  0.0971,
         0.2717, -0.0007, -0.2765,  0.3127,  0.0387,  0.0097, -0.0135,  0.0609,
        -0.3462,  0.1779,  0.3245, -0.0028,  0.0162, -0.0771, -0.0212, -0.0601,
        -0.0312, -0.0744, -0.4155,  0.1605,  0.4971,  0.0330])

In [ ]:

learn.predict(L(full_x))

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()


In [ ]:
#export
@delegates(Learner.__init__)
def text_regression_learner(dbunch, arch, seq_len=72, config=None, pretrained=True, drop_mult=0.5,
                            lin_ftrs=None, ps=None, **kwargs):
    "Create a `Learner` with a text classifier from `data` and `arch`."
    vocab = _get_text_vocab(dbunch)
    model = get_text_classifier(arch, len(vocab), len(dbunch.train_ds[0][1]), seq_len=seq_len, config=config,
                                drop_mult=drop_mult, lin_ftrs=lin_ftrs, ps=ps)
    meta = _model_meta[arch]
    learn = TextLearner(dbunch, model, loss_func=MSELossFlat(), splitter=meta['split_clas'], **kwargs)
    if pretrained:
        if 'url' not in meta:
            warn("There are no pretrained weights for that architecture yet!")
            return learn
        model_path = untar_data(meta['url'], c_key='model')
        fnames = [list(model_path.glob(f'*.{ext}'))[0] for ext in ['pth', 'pkl']]
        learn = learn.load_pretrained(*fnames, model=learn.model[0])
        learn.freeze()
    return learn